In [ ]:
# Essential libs
import pandas as pd
import openpyxl # to read xlsx
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels
import scipy.stats
# Chisquare
from scipy.stats import chisquare
# ANOVA
from scipy.stats import f_oneway
# Wilcoxon
from scipy.stats import wilcoxon
from scipy.stats import ranksums
import pingouin as pg 
# Normality testing
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import anderson
# Trim_mean function
from scipy.stats import trim_mean
# Hypothesis testingg
from scipy.stats import t
from scipy.stats import ttest_1samp
from scipy.stats import ttest_ind_from_stats 
# Sampling
import random
from random import sample
from sklearn.utils import resample
# Linear regression
from sklearn.linear_model import LinearRegression


In [ ]:
# This is used to block warning messages
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")
# Or ignore specific warnings by category (e.g., FutureWarnings)
warnings.filterwarnings("ignore", category=FutureWarning)
# This is used to edit image
from IPython.display import Image
from IPython.core.display import HTML 

In [ ]:
df  = pd.read_excel(r'C:\Users\Admin\Dognition\_1e0dffe5f998bae66fea694bbcc0fd99_dognition_data_no_aggregation_with_zip_code_correction.xlsx')

In [ ]:
df.columns 

# 1. Normal distribution

> How to sample a normal distribution and plot it:

In [ ]:
# Sample and plot normal dist
np.random.seed(31)
x = np.random.randn(1000,)
sns.kdeplot(x)

> To examine normality of a data set, you can:
> * Visualize the data and see if it has the bell-curved shape
> * Visualize the data on qqplot. If the data is heavily distributed around the diagonal line, then it's likely to have a normal distribution
> * Use statistical test (recommnended). There are several tests to use, such as Shapiro, or Anderson.

In [ ]:
# Examine using qqplot
ax = qqplot(x, line='s')
plt.show(ax)

> As you can see, this data is normally distributed and when plot it on qqplot, data points are centered around the red line.

In [ ]:
# Shapiro test for normality
shapiro(x)

> The null hypothesis of shapiro test is: Data follows normal distribution. 
> Pvalue of the test is less than 1-5-10%, meaning the null hypothesis is accepted

In [ ]:
# Anderson
print(anderson(x).significance_level,
      anderson(x).critical_values,
      anderson(x).statistic)

> Anderson test's H0 statese that the data follow a particular distribution. As default, the examined distribution is the Gaussian (normal)
> 
> To check the result:
> * Choose desired significant level (it is the alpha)
> * Get critical value assigned with that level of significance
> * Compare test statistic with the critical value. If Test statistic < Critical value, then accept Null hypothesis and vice versa.

# 2. Descriptive Analysis

## Calculation

> There are several main aspects of the data that we hope to describe: 
> * <strong> Location.</strong> location shows the value that the data center around
> * <strong>Dispersion.</strong> Dispersion shows how far the data is compared to its location
> * <strong>Correlation.</strong> Correlation shows the strength and direction of two variables' relationship. 

In [ ]:
###### LOCATION #######
# data
data = df['Rank_by_DogID'].dropna()
# mean
print('mean', data.mean())
# trim-mean (from scipy.stats import trim_mean)
print('trim-mean', trim_mean(data, 0.1))
# median
print('median', data.median())

In [ ]:
###### DISPERSION ######
# data
data = df['Rank_by_DogID'].dropna()
# standard deviation, variance
print('standard deviation', data.std(), "variance", data.std() ** 2)
# quantile
data1 = data.values
quantiles = [q/10 for q in range(2,10,2)]
for quan in quantiles:
    print(f"quantile {quan} ", np.quantile(data1, quan))

In [ ]:
###### CORRELATION #######
# data 
data_corre = df.dropna(subset=['Sign_in_Count', 'Rank_by_UserID'])
x = data_corre['Sign_in_Count']
y = data_corre['Rank_by_UserID']
# covariance
cov_matrix = pd.DataFrame(np.cov(x,y), index=('x','y'), columns=['x','y'])
# correlation coefficient
corrcoef_matrix = pd.DataFrame(np.corrcoef(x,y), index=('x','y'), columns=['x','y'])
# display result
display(
    "cov_matrix: ", cov_matrix,
    "covariance of x - y: ", cov_matrix.loc["x","y"],
    'corrcoef_matrix: ', corrcoef_matrix,
    "corrcoef of x - y: ", corrcoef_matrix.loc["x","y"]
)

In [ ]:
##### PROBABILITY (Distribution) #####
# data
data = df['Rank_by_DogID'].dropna()
data_dist = (data.values / sum(data)) * 100

print('THIS IS THE PROB TABLE')
pd.DataFrame(
    {"value" : list(data.values),
    "probability" : list(data_dist)}
)

In [ ]:
# describe function
df[['Rank_by_UserID','Sign_in_Count']].describe()

## Visualization

In [ ]:
# The boxplot
sns.boxplot(df['Weight'])

In [ ]:
# The violin
sns.violinplot(df['Weight'])

In [ ]:
# Categorical data visualization
sns.catplot(
    data = df[['Gender','Weight']],
    x = 'Gender',
    y = 'Weight',
    kind = 'violin'
)

In [ ]:
# scatter plot
sns.scatterplot(
    data = pd.DataFrame(np.random.randn(1000,2),columns=["x","y"]),
    x = "x",
    y = 'y',
)

In [ ]:
# Heat map
data = df[['Rank_by_UserID','Sign_in_Count','Max_Dogs']].dropna()
corre_matrix = np.corrcoef(data.T)

display(corre_matrix)
sns.heatmap(corre_matrix)

In [ ]:
# pairplot
data = pd.DataFrame(np.random.randn(100,3),columns=["x","y","z"])
sns.pairplot(data, kind ="scatter")

# Range Estimation - Confidence interval

> With the observed data, we can estimate the value range that the data' parameters (mean, standard deviation,...) fall into. 
>
> The steps for calculation are:
> * Get the number of observation (n)
> * Get the location (mean)
> * Get the dispersion (standard deviation)
> * Get the standard error by dividing the Standard deviation with square of n
> * Get the t value for the desired significant level
> * Combine those value using this formula

![The San Juan Mountains are beautiful!](\Image\Confidence-Interval-Formula.jpg "San Juan Mountains")

In [ ]:
# Data
data = df['Rank_by_DogID'].dropna()
# Number of observation
n = len(data)
# Mean
X_bar = data.mean()
# Standard deviation
StanD = np.std(data)
# Standard error
StanE = StanD / np.sqrt(n) # sigma/sqrt(n)
# t-Value
t_val = t.ppf(0.95, df=n-1) # ppf take first argument as the quantile

In [ ]:
# Compute confidence interval
con_inv = (X_bar - t_val*StanE, X_bar + t_val*StanE)
con_inv

# Basic Hypothesis testing

> <p> 
> Hypothesis testing is a broad topic with numerous existing methods. <br>
> In this section, we cover the most basic yet widely used test, which is T test for mean of one and two samples. <br>
> <ul>
> <strong> Why examine the mean</strong>: Most of the time, the average value of data sets can give enough information for us to check our hypotheses.  <br>
> <strong> Why using T value</strong>: Student distribution (t) has the same bell-curved shape as the Gaussian distribution. However, t distribution is <br>
> more flexible since we can use its value for data that has small size or when we does not know the sigma of population. Moreover, when observation size <br>
> increase, the t distribution is approximately identical to the Gaussian distribution.
>  <br>
> </ul>
> </p>

In [ ]:
##### ONE SAMPLE T-TEST #####
data = df['Rating'].dropna().values
# H1: Mean of Rating > 2.5
ttest_1samp(data, popmean = 2.5, alternative="greater")

In [30]:
##### TWO SAMPLES T-TEST #####
rating_of_tested = df[df['DNA_Tested'] == 1]['Rating'].dropna().values
rating_of_no_tested = df[df['DNA_Tested'] == 0]['Rating'].dropna().values
# H1: Rating of DNA_Tested > Rating of DNA_Not_Tetsted
ttest_ind_from_stats(
    mean1 = rating_of_tested.mean(), std1 = np.std(rating_of_tested), nobs1 = len(rating_of_tested), 
    mean2 = rating_of_no_tested.mean(), std2 = np.std(rating_of_no_tested), nobs2 = len(rating_of_no_tested),
    alternative = 'greater'
)

Ttest_indResult(statistic=1.0686681511758527, pvalue=0.14388337177925895)

# Chisquare Tests

<p>
Chisquare tests are mostly used to confirm the distribution of a data set. <br>
The H0 in chisquare tests is that the data <strong> does not </strong> follow a particular distribution. <br>
The chisquare tests are used in problems such as:
<ul>
- Confirming the data has a normal distribution, or any other interested common distribution. <br>
- Examine the relationship between two categorical variables. <br>
</ul>
</p>

## Distribution Hypothesis test (Goodness-of-fit Test)

In [ ]:
chi = df.groupby(['DNA_Tested','Subscribed']).agg({'User_ID':'count'}).reset_index()
chi

In [ ]:
# rename
chi = chi.rename(columns={'User_ID':'Obser_count'})
# take expected distribution
chi['Exp_dist'] = 1/ len(chi['Subscribed'].unique())
# generate a column with the total sample of tested and not tested DNA - dog
chi['Total_sample'] = chi.groupby('DNA_Tested')['Obser_count'].transform(sum)
# Get expected count
chi['Exp_count'] = chi['Total_sample'] * chi['Exp_dist']
# Get observed distribution
chi['Obs_dist'] = chi['Obser_count'] / chi['Total_sample']
# Take only the distribution columns to compare
chi[['DNA_Tested','Subscribed', 'Obs_dist','Exp_dist']]
# print(0.630765 - 0.369235, 0.592443- 0.407557) # compare the diff

In [ ]:
# Test the hypothesis
f_obs = chi['Obser_count'].values
f_exp = chi['Exp_count'].values
chisquare(f_obs,f_exp)

# Analysis of Variance (ANOVA)

> <p>
> The Analysis of Variance is unique in terms of enabling hypothesis testing for more than two samples.
> </p>

## One-way ANOVA

In [ ]:
df.dropna(subset=['Rank_by_DogID']).groupby('Dimension')['Rank_by_DogID'].mean()

In [ ]:
list_of_sample = []
for dimension in df['Dimension'].dropna().unique():
    sam = df[df['Dimension'] == dimension]['Rank_by_DogID'].dropna()
    list_of_sample.append(np.array(sam))
list_of_sample[1]

In [ ]:
f_oneway(list_of_sample[0],
        list_of_sample[1],
        list_of_sample[2],
        list_of_sample[3],
        list_of_sample[4],
        list_of_sample[5],
        list_of_sample[6],
        list_of_sample[7],
        list_of_sample[8])

> Welch ANOVA is used when prerequisites in ANOVA is not met.
>
> Those prerequisites are: Groups have equal variance, Normal distribution, Large enough sample.

In [ ]:
# Welch ANOVA
pg.welch_anova(dv= 'Rank_by_DogID', between= 'Dimension', data= df.dropna(subset=['Dimension','Rank_by_DogID']))

> Note: ANOVA is typically used to examine the difference between means of groups, which are divided by one categorical variable.
>
> Regression with dummy variable can replace ANOVA if wanted. However Regression with dummy variable is more suitable when there are more than one categorical variables and when the purpose is to examine the relationship between factors.

# Non-parametric Methods (Wilconxon test)

> When assumption about the normality of data is not met, one common non-parametric method which is the wilconxon test is used. <br>
> Wilconxon test ignore mean and variance of the data. This method cares only about <strong> the median </strong>, which is not affected by the distribution of data.

## One sample / Two paired samples

In [ ]:
# One sample - Rating
data = df['Rating'].dropna().values
print(np.median(data))
print(wilcoxon(data-2.5, alternative='greater'))

## Two independent samples

In [ ]:
male = df[df['Gender'] == 'male']['Rating'].dropna().values
female = df[df['Gender'] == 'female']['Rating'].dropna().values

print(len(male),len(female))
print(np.median(male) , np.median(female))

sns.boxplot([male,female])

ranksums(x=male,y=female,alternative='greater')

# Bootstraping / Permutation

> The last thing we will cover is sampling methods, particularly bootstraping and permutation

## Bootstraping

**Brief introduction about bootstraping:**

* Its a Resampling method.
* Estimates sampling distribution by repeatedly drawing samples with replacement.
* Useful when population distribution is unknown or sample size is limited.
* Calculates confidence intervals, standard errors, and assesses uncertainty

In [ ]:
# Linear regresion on "Sign_in_Count" -> "Rank_by_UserID"
sign_in_rank = df[['Sign_in_Count','Rank_by_UserID']].dropna()

In [ ]:
# resample function
ori = sign_in_rank[['Sign_in_Count','Rank_by_UserID']]
sampled = resample(sign_in_rank[['Sign_in_Count','Rank_by_UserID']])
display('ori: ', ori.mean(), 'sampled: ', sampled.mean())

In [ ]:
def boostraping_for_sign_rank_problem(data, times):
    result_list = []
    for boostrap_time in range(times):
        # sampling data
        sample_data = resample(data)
        model = LinearRegression()
        model.fit(sample_data[['Sign_in_Count']],sample_data[['Rank_by_UserID']])
        result_list.append(model.coef_[0][0])
    return np.array(result_list)

In [ ]:
res_ar = boostraping_for_sign_rank_problem(sign_in_rank, times = 100)
# see its distribution
sns.kdeplot(res_ar)
# calculate the average value from every sample
print("Average coeff: ", np.mean(res_ar))
print("Original coeff: ", LinearRegression().fit(sign_in_rank[['Sign_in_Count']],sign_in_rank[['Rank_by_UserID']]).coef_[0][0])

## Permutation

**Brief introduction about permutation:**

* Its a Non-parametric statistical method.
* Assesses significance without assuming specific population distributions.
* Involves shuffling or rearranging data points to create a null distribution.
* Valuable for non-normal data and when parametric assumptions don't hold.

In [ ]:
# DNA tested has higher rating than DNA not tested
rating_dnatest = df[['Rating','DNA_Tested']].dropna()

In [ ]:
# check for normality
sns.kdeplot(rating_dnatest.Rating)
shapiro(rating_dnatest[['Rating']]) # conclusion: not normal

In [ ]:
# observed test statistic
mean_test = rating_dnatest[rating_dnatest['DNA_Tested'] == 1]['Rating'].mean()
mean_not_test = rating_dnatest[rating_dnatest['DNA_Tested'] == 0]['Rating'].mean()
test_statistic = mean_test - mean_not_test
print('mean_test: ',mean_test,'mean_not_test: ',mean_not_test, 'test_statistic: ', test_statistic)

In [ ]:
def permutating(data, n_group1, times):
    permutation_result = []
    for permu_time in range(times):
        sample_data = data.sample(len(data))
        sample_group1 = sample_data[:n_group1]
        sample_group2 = sample_data[n_group1:]
        # get value to get from sample
        sample_result = sample_group1.mean() - sample_group2.mean()
        permutation_result.append(sample_result)
    return np.array(permutation_result)

In [ ]:
n_group1 = len(rating_dnatest[rating_dnatest['DNA_Tested'] == 1]['Rating'])
df_res = permutating(rating_dnatest[['DNA_Tested']], n_group1, 1000)

In [ ]:
sns.kdeplot(df_res)

In [ ]:
def get_p_value(res_permu, observed_test_statistic):
    count_extreme_than_stat = len(res_permu[res_permu > observed_test_statistic])
    p_value = count_extreme_than_stat / len(res_permu) 
    return p_value

In [ ]:
get_p_value(df_res, test_statistic)